
# RAG with Chroma

1. 문서 읽기
2. 문서를 청크로 나눈다.
3. 문서를 임베딩한다. -> 벡터 데이터베이스에 저장
4. 질문이 있을때 벡터데이터 베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

## 1. 문서 읽기

In [59]:
%pip install -qU docx2txt langchain-community langchain-text-splitters unstructured python-docx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
#%pip uninstall -y pdfminer pdfminer.six
#%pip install pdfminer.six
#%pip install pi_heif
#%pip install unstructured_inference
#%pip install pdf2image


from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 읽기
loader = UnstructuredPDFLoader("./data/2025 공무원보수 등의 업무지침(인사혁신처 예규 제192호)-맞춤형복지.pdf"
    , mode="elements"
    , strategy="auto"
    , languages=["kor"]
    )

# chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
document_list = loader.load_and_split(text_splitter)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [16]:
document_list

[Document(metadata={'source': './data/2025 공무원보수 등의 업무지침(인사혁신처 예규 제192호)-맞춤형복지.pdf', 'coordinates': {'points': ((np.float64(305.0), np.float64(302.0)), (np.float64(305.0), np.float64(1836.0)), (np.float64(1559.0), np.float64(1836.0)), (np.float64(1559.0), np.float64(302.0))), 'system': 'PixelSpace', 'layout_width': 1795, 'layout_height': 2442}, 'filetype': 'application/pdf', 'languages': ['kor'], 'last_modified': '2025-12-06T19:43:31', 'page_number': 1, 'file_directory': './data', 'filename': '2025 공무원보수 등의 업무지침(인사혁신처 예규 제192호)-맞춤형복지.pdf', 'category': 'UncategorizedText', 'element_id': 'c31e31b1797a459e32d5e012cbfdbb2d'}, page_content=", 00) 2 ~ 0) 작 0) [9 (00) 0) 때 기 2 < 0) [9 [9 [9 22 [0)) 0 / ' < < 9) ㅇㅇ ㅇㅇ 9) < 0) ㅇㅇ 관 아쟁 ~ 620 벼 (대 이 은 은 0 내 개 몰 1 010 메 께 에 0 0 1 | 1 16 1 그 회 : : _ 호 : ! : 교 피 때 더 020 001 : 뜨지 고 # 0 히 에 < 이 | 에 호 도 바 이 < 제 월 츠 록 으 즈 히 하 하 - ㅁ ㅠ 모 쁘 매 버 버 버 명 표피 을 할 호 휴대 85 마 ~ 고 | < 과 후 도 인 

## embedding
### sentence-transformers

In [42]:
%pip install sentence-transformers langchain-huggingface

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 16.7 MB/s eta 0:00:00a 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 14.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.4 MB/s eta 0:00:0

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

# embedding
embeddings = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')

## Chroma DB 생성후 파일로 저장

In [21]:
%pip install langchain_community


from langchain_chroma import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata

document_list = filter_complex_metadata(
    document_list
)

database = Chroma.from_documents(
    document_list, 
    embeddings,
    collection_name="gwp2025_pdf",
    persist_directory="./chroma_pdf_db",
    )

database    


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Chrmoma DB 로드

In [ ]:
from langchain_chroma import Chroma

database = Chroma(
    collection_name="gwp2025_pdf",
    persist_directory="./chroma_pdf_db",
    embedding_function=embeddings,
)

database 

NameError: name 'embeddings' is not defined

In [22]:
query = '맞춤형 복지 제한되는 경우는 무엇인가요?'

retd_docs = database.similarity_search(query)

retd_docs

[Document(id='e99ea576-4c39-478a-ad7c-783d99181ddc', metadata={'page_number': 36, 'category': 'ListItem', 'last_modified': '2025-12-06T19:43:31', 'element_id': '608024245be25c072b40f8c021a72b6b', 'source': './data/2025 공무원보수 등의 업무지침(인사혁신처 예규 제192호)-맞춤형복지.pdf', 'filetype': 'application/pdf', 'parent_id': '3b3d928c062dfbbfaa362ea0d40e3c10', 'filename': '2025 공무원보수 등의 업무지침(인사혁신처 예규 제192호)-맞춤형복지.pdf', 'file_directory': './data'}, page_content='1. 맞춤형 복지'),
 Document(id='6502b330-1df8-4607-b498-10c92160465a', metadata={'category': 'Title', 'filename': '2025 공무원보수 등의 업무지침(인사혁신처 예규 제192호)-맞춤형복지.pdf', 'filetype': 'application/pdf', 'page_number': 7, 'source': './data/2025 공무원보수 등의 업무지침(인사혁신처 예규 제192호)-맞춤형복지.pdf', 'element_id': 'e769a76720d09ba749bcace9ed8601fc', 'last_modified': '2025-12-06T19:43:31', 'file_directory': './data'}, page_content='나. 기

In [23]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-2.5-flash")



In [24]:
#%pip install langchain_classic
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

In [8]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [28]:
#from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = database.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnableParallel(
        context=retriever,
        question=RunnablePassthrough(),    
    )
    | prompt
    | llm
)    

ai_message = rag_chain.invoke("맞춤형복지 복지점수의 구성?")

print(ai_message)


제공된 컨텍스트에는 맞춤형복지 복지점수의 구체적인 구성에 대한 정보가 없습니다. 복지 형평성과 분포를 고려한다는 기준만 언급되어 있습니다.


In [13]:
#%pip install -U langchain langchain-core langchain-community
%pip show langchain

Name: langchain
Version: 1.1.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Users/hunchulchoi/.pyenv/versions/3.12.12/envs/inflearn-llm-application/lib/python3.12/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.
